In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVR
from sklearn import linear_model

# Data Cleaning

In [2]:
data = pd.read_excel('baseball_LL.xlsx')

In [3]:
#cleaned the data
players = []
for name in data.Name:
    player = name.replace(' ','')
    player = player.replace('\xa0', '')
    players+=[player]
data['Name'] = players

In [4]:
stats = data.drop(['GP-GS', '2B', '3B', 'HR', 'RBI','BB', 'HBP','SO', 'GDP', 'SF', 'SH', 'SB-ATT'], axis=1)

In [5]:
convert = ['AVG', 'OPS', 'AB', 'R', 'H', 'TB', 'SLG%',  'OB%']
for val in convert:
    stats[val]=stats[val].astype(float)

In [6]:
RpAB = []
HpAB = []
TBpAB = []
for index, row in stats.iterrows():
    if row.AB!=0:
        RpAB += [row.R/row.AB]
        HpAB += [row.H/row.AB]
        TBpAB += [row.TB/row.AB]
    else:
        RpAB += [0]
        HpAB += [0]
        TBpAB += [0]
stats['RpAB']=RpAB
stats['HpAB'] = HpAB
stats['TBpAB'] = TBpAB

In [7]:
#create four dataframes

#players who have played at least two seasons:
two_seasons = pd.DataFrame(columns = stats.columns)
#players who have played at least three seasons:
three_seasons = pd.DataFrame(columns = stats.columns)
#players who have played at least four seasons:
four_seasons = pd.DataFrame(columns = stats.columns)

In [9]:
#add the data to the dataframes based on above specifications

for player in stats.Name.unique():
    if stats[stats.Name==player].shape[0]>1:
        two_seasons = two_seasons.append(stats[stats.Name==player].tail(2))
    if stats[stats.Name==player].shape[0]>2:
        three_seasons = three_seasons.append(stats[stats.Name==player].tail(3))
    if stats[stats.Name==player].shape[0]==4:
        four_seasons = four_seasons.append(stats[stats.Name==player])

In [10]:
#reshaped those dataframes 

cols_2 = ['Name','First_Year','School','AVG1', 'OPS1', 'AB1', 'R1', 'H1', 'TB1', 'SLG1','OB1','RpAB1', 'HpAB1', 'TBpAB1',
         'AVG2', 'OPS2', 'AB2', 'R2', 'H2', 'TB2', 'SLG2', 'OB2', 'RpAB2', 'HpAB2', 'TBpAB2']
two_years = pd.DataFrame(columns = cols_2)

cols_3 = ['Name','First_Year','School','AVG1', 'OPS1', 'AB1', 'R1', 'H1', 'TB1', 'SLG1', 'OB1','RpAB1', 'HpAB1', 'TBpAB1', 
         'AVG2', 'OPS2', 'AB2', 'R2', 'H2', 'TB2', 'SLG2', 'OB2','RpAB2', 'HpAB2', 'TBpAB2', 
         'AVG3', 'OPS3', 'AB3', 'R3', 'H3', 'TB3', 'SLG3', 'OB3','RpAB3', 'HpAB3', 'TBpAB3']
three_years = pd.DataFrame(columns = cols_3)

cols_4 = ['Name','First_Year','School','AVG1', 'OPS1', 'AB1', 'R1', 'H1', 'TB1', 'SLG1', 'OB1','RpAB1', 'HpAB1', 'TBpAB1', 
         'AVG2', 'OPS2', 'AB2', 'R2', 'H2', 'TB2', 'SLG2', 'OB2','RpAB2', 'HpAB2', 'TBpAB2', 
         'AVG3', 'OPS3', 'AB3', 'R3', 'H3', 'TB3', 'SLG3', 'OB3','RpAB3', 'HpAB3', 'TBpAB3',
         'AVG4', 'OPS4', 'AB4', 'R4', 'H4', 'TB4', 'SLG4', 'OB4','RpAB4', 'HpAB4', 'TBpAB4']
four_years = pd.DataFrame(columns = cols_4)

In [11]:
#fill in those new dataframes

for player in two_seasons.Name.unique():
    player_stats = two_seasons[two_seasons.Name==player]
    yr_school = player_stats.values[1][9:11]
    s2 = player_stats.values[0][1:9].tolist()+player_stats.values[0][11:].tolist()
    s1 = player_stats.values[1][1:9].tolist()+player_stats.values[1][11:].tolist()
    player_data = np.concatenate([[player],yr_school,s1,s2])
    two_years.loc[len(two_years)] = player_data

In [12]:
for player in three_seasons.Name.unique():
    player_stats = three_seasons[three_seasons.Name==player]
    yr_school = player_stats.values[2][9:11]
    s3 = player_stats.values[0][1:9].tolist()+player_stats.values[0][11:].tolist()
    s2 = player_stats.values[1][1:9].tolist()+player_stats.values[1][11:].tolist()
    s1 = player_stats.values[2][1:9].tolist()+player_stats.values[2][11:].tolist()
    player_data = np.concatenate([[player],yr_school,s1,s2,s3])
    three_years.loc[len(three_years)] = player_data

In [13]:
for player in four_seasons.Name.unique():
    player_stats = four_seasons[four_seasons.Name==player]
    yr_school = player_stats.values[3][9:11]
    s4 = player_stats.values[0][1:9].tolist()+player_stats.values[0][11:].tolist()
    s3 = player_stats.values[1][1:9].tolist()+player_stats.values[1][11:].tolist()
    s2 = player_stats.values[2][1:9].tolist()+player_stats.values[2][11:].tolist()
    s1 = player_stats.values[3][1:9].tolist()+player_stats.values[3][11:].tolist()
    player_data = np.concatenate([[player],yr_school,s1,s2,s3,s4])
    four_years.loc[len(four_years)] = player_data

# Two Season Data Analysis

In [14]:
drop_all2 = ['Name', 'First_Year', 'School','AVG2', 'OPS2', 'AB2', 'R2', 'H2', 'TB2', 'SLG2', 'OB2','RpAB2', 'HpAB2', 'TBpAB2']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(two_years.drop(drop_all2, axis=1),two_years['SLG2'], test_size=0.3, random_state=100)
clf = RandomForestRegressor(n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracies:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Accuracies:
Mean Absolute Error: 0.12616176075268817
Mean Squared Error: 0.03000993153053315
Root Mean Squared Error: 0.17323374824361779


# Three Season Data Analysis

In [16]:
drop_all3 = ['Name', 'First_Year', 'School','AVG3', 'OPS3', 'AB3', 'R3', 'H3', 'TB3', 'SLG3', 'OB3','RpAB3', 'HpAB3', 'TBpAB3']

In [17]:
def find_trend_2(data, stat, y1, y2):
    stat_1 = stat+'1'
    stat_2 = stat+'2'
    result = []
    for index, row in data.iterrows():
        result+=[round(float(row[stat_2])-float(row[stat_1]),3)]
    return(result)

In [18]:
stat_names = ['AVG','OPS','AB','R','H','TB','SLG','OB','RpAB', 'HpAB', 'TBpAB']
for stat in stat_names:
    trend = find_trend_2(three_years, stat, '1', '2')
    colname = stat+'trend'
    three_years[colname]=trend

In [19]:
X_train, X_test, y_train, y_test = train_test_split(three_years.drop(drop_all3, axis=1),three_years['SLG3'], test_size=0.3, random_state=100)
clf = RandomForestRegressor(n_estimators=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracies:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Accuracies:
Mean Absolute Error: 0.10847138888888888
Mean Squared Error: 0.01860755202938271
Root Mean Squared Error: 0.13640950124306853


# Four Seasons Data Analysis

In [20]:
drop_all4= ['Name', 'First_Year', 'School','AVG4', 'OPS4', 'AB4', 'R4', 'H4', 'TB4', 'SLG4',  'OB4','RpAB4', 'HpAB4', 'TBpAB4']

In [21]:
stat_names = ['AVG','OPS','AB','R','H','TB','SLG','OB','RpAB', 'HpAB', 'TBpAB']
for stat in stat_names:
    trend = find_trend_2(four_years, stat, '1', '2')
    colname = stat+'trend1'
    four_years[colname]=trend
for stat in stat_names:
    trend = find_trend_2(four_years, stat, '2', '3')
    colname = stat+'trend2'
    four_years[colname]=trend
for stat in stat_names:
    trend = find_trend_2(four_years, stat, '1', '3')
    colname = stat+'trend3'
    four_years[colname]=trend

In [22]:
X_train, X_test, y_train, y_test = train_test_split(four_years.drop(drop_all4, axis=1),four_years['AVG4'], test_size=0.3, random_state=100)
clf = RandomForestRegressor(n_estimators = 25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracies:')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Accuracies:
Mean Absolute Error: 0.06974511627906975
Mean Squared Error: 0.009787474381395347
Root Mean Squared Error: 0.0989316652108684


# Predictions for Rochester Players

In [23]:
predictions = pd.read_excel('predictions.xlsx')
predictions.columns = ['Player Name', 'AVG', 'OB', 'SLG', 'OPS', 'Class Year']

In [37]:
#Brian, Harper, Matzat, Rende, Luke
sophs = ['McKinsey,Brian','Sy,Harper','Matzat,Jacob','Rende,Joseph','Piontek,Luke']
jrs = ['Bankovich,Drew','Pickering,Steve']
srs = ['Craig,Aaron','Rieth,David','Hertz,Jake','Trombley,Kyle','McNabb,Ryland','Miraz,Zach']

sophs_data = stats.loc[stats['Name'].isin(sophs)]
jrs_data = two_years.loc[two_years['Name'].isin(jrs)]
srs_data = three_years.loc[three_years['Name'].isin(srs)]

In [38]:
stat_names = ['AVG','OPS','AB','R','H','TB','SLG','OB','RpAB', 'HpAB', 'TBpAB']
for stat in stat_names:
    trend = find_trend_2(jrs_data, stat, '1', '2')
    colname = stat+'trend'
    jrs_data[colname]=trend
    
for stat in stat_names:
    trend = find_trend_2(srs_data, stat, '2', '3')
    colname = stat+'trend2'
    srs_data[colname]=trend
for stat in stat_names:
    trend = find_trend_2(srs_data, stat, '1', '3')
    colname = stat+'trend3'
    srs_data[colname]=trend

C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [26]:
def run_rf(train, drop_cols, years, pred_cols, player_data):
    df = pd.DataFrame()
    df['Name']=player_data.Name
    X_train = train.drop(drop_cols, axis=1)
    for pred in pred_cols:
        metric = pred+years   #e.g. AVG2
        y_train = train[metric] 
        clf = RandomForestRegressor(n_estimators=25)
        clf.fit(X_train, y_train)
        if int(years)>2:
            preds = clf.predict(player_data.drop(['Name','First_Year','School'],axis=1))
        else:
            preds = clf.predict(player_data.drop(['Name','Year','School'],axis=1))
        df[pred]=preds
    return (df)

In [27]:
def run_lm(train, drop_cols, years, pred_cols, player_data):
    df = pd.DataFrame()
    df['Name']=player_data.Name
    X_train = train.drop(drop_cols, axis=1)
    for pred in pred_cols:
        metric = pred+years   #e.g. AVG2
        y_train = train[metric] 
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        if int(years)>2:
            preds = regr.predict(player_data.drop(['Name','First_Year','School'],axis=1))
        else:
            preds = regr.predict(player_data.drop(['Name','Year','School'],axis=1))
        df[pred]=preds
    return(df)

In [40]:
#make predictions for roc players using rf
pred_cols = ['AVG','OPS','SLG','OB']
preds_rf = run_rf(two_years, drop_all2,  '2', pred_cols,sophs_data)

preds_rf = preds_rf.append(run_rf(three_years,drop_all3,'3',pred_cols,jrs_data))

preds_rf = preds_rf.append(run_rf(four_years,drop_all4,'4',pred_cols,srs_data))
preds_rf

,Name,AVG,OPS,SLG,OB
1,"Matzat,Jacob",0.31060,0.77924,0.31900,0.39584
8,"Sy,Harper",0.32168,0.81396,0.43768,0.40152
10,"Rende,Joseph",0.26528,0.54748,0.30992,0.30264
13,"McKinsey,Brian",0.22980,0.71288,0.27448,0.38164
16,"Piontek,Luke",0.24304,0.77616,0.44572,0.37356
4,"Pickering,Steve",0.33192,0.75856,0.37892,0.39248
7,"Bankovich,Drew",0.22788,0.71640,0.30028,0.37760
1,"Rieth,David",0.31492,0.89780,0.46248,0.41172
2,"Hertz,Jake",0.28944,0.89344,0.43480,0.40556
4,"Trombley,Kyle",0.27524,0.76856,0.36860,0.36276


In [41]:
#make predictions for roc players using lm
preds_lm = run_lm(two_years, drop_all2,  '2', pred_cols,sophs_data)

preds_lm = preds_lm.append(run_lm(three_years,drop_all3,'3',pred_cols,jrs_data))

preds_lm = preds_lm.append(run_lm(four_years,drop_all4,'4',pred_cols,srs_data))

preds_lm

,Name,AVG,OPS,SLG,OB
1,"Matzat,Jacob",0.287031,0.716868,0.337854,0.379014
8,"Sy,Harper",0.286270,0.728742,0.354013,0.374729
10,"Rende,Joseph",0.255824,0.642826,0.303614,0.339212
13,"McKinsey,Brian",0.234327,0.619518,0.297799,0.321719
16,"Piontek,Luke",0.208404,0.563838,0.260766,0.303072
4,"Pickering,Steve",0.342810,0.835764,0.420562,0.415202
7,"Bankovich,Drew",0.271951,0.697800,0.342177,0.355622
1,"Rieth,David",0.338021,0.926857,0.494842,0.432056
2,"Hertz,Jake",0.273755,0.716025,0.382784,0.333201
4,"Trombley,Kyle",0.207132,0.603302,0.295789,0.307512


In [42]:
def eval_preds(preds_model,preds_coach):
    df_diff = pd.DataFrame()
    
    df_diff['Name'] = list(preds_model.Name)
    for stat in pred_cols:
        this_stat = []
        for name in df_diff.Name:
            val = round(float(preds_coach[stat][preds_coach['Player Name']==name])-float(preds_model[stat][preds_model.Name==name]),3)
            pct = round(100*(float(preds_coach[stat][preds_coach['Player Name']==name])-float(preds_model[stat][preds_model.Name==name]))/float(preds_model[stat][preds_model.Name==name]),2)
            entry = str(val)+' ('+str(pct)+'%)'
            this_stat+=[entry]
        df_diff[stat]=this_stat
    return(df_diff)

In [43]:
eval_preds(preds_lm,predictions)
#eval_preds(preds_rf,predictions)

,Name,AVG,OPS,SLG,OB
0,"Matzat,Jacob",0.033 (11.49%),0.053 (7.41%),0.082 (24.31%),-0.029 (-7.66%)
1,"Sy,Harper",0.024 (8.29%),0.121 (16.64%),0.046 (12.99%),0.075 (20.09%)
2,"Rende,Joseph",0.094 (36.81%),0.267 (41.56%),0.146 (48.21%),0.121 (35.61%)
3,"McKinsey,Brian",0.086 (36.56%),0.19 (30.75%),0.112 (37.68%),0.078 (24.33%)
4,"Piontek,Luke",0.072 (34.35%),0.169 (30.0%),0.139 (53.39%),0.03 (9.87%)
5,"Pickering,Steve",-0.078 (-22.7%),-0.081 (-9.66%),-0.041 (-9.64%),-0.04 (-9.68%)
6,"Bankovich,Drew",-0.083 (-30.5%),-0.148 (-21.18%),0.008 (2.29%),-0.156 (-43.76%)
7,"Rieth,David",0.072 (21.29%),0.093 (10.05%),0.105 (21.25%),-0.012 (-2.79%)
8,"Hertz,Jake",0.061 (22.37%),0.084 (11.73%),0.017 (4.5%),0.067 (20.05%)
9,"Trombley,Kyle",0.043 (20.7%),-0.028 (-4.69%),0.004 (1.42%),-0.033 (-10.57%)


## Don't need
two_years.to_csv('two_years.csv')
three_years.to_csv('three_years.csv')
four_years.to_csv('four_years.csv')